**Algorithms and Programming Project:**
**Subject 2 :Steganography**

Group members:
Maryam Forouhari,
Joshua Dakim

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def steganography(num:int=2,boolinput:bool=True):

    """
    Hides a boolean value inside the least significant bit of an integer.

    Parameters:
    num       : integer where the data will be hidden
    boolinput : boolean value to hide (True -> 1, False -> 0)

    Returns:
    An integer with the boolean value encoded in its LSB.
    """

    # Convert boolean input to integer (True -> 1, False -> 0)

    seccode=int(boolinput)

    # Get the current least significant bit of the number

    last_bit=ret_LSB(num)

    # If the current LSB is different from the bit to hide,
    # flip the last bit using XOR with 1

    if last_bit!=seccode:
        newnum=num^1


  # If the LSB is already correct, keep the number unchanged
    else:
        newnum=num


    return newnum


# return last bit of an integer

def ret_LSB(num:int=1000013):
    """
    Returns the least significant bit (LSB) of an integer.
    If the number is even, it returns 0.
    If the number is odd, it returns 1.
    """

    last_bit=num%2
    return last_bit



**Question 3:** Hide and retrieve a binary image inside a grayscale image

In [ ]:
try:
  # Load grayscale cover image
  im1 = cv.imread("pic1.jpg", cv.IMREAD_GRAYSCALE)
  # Load binary image
  im2 = cv.imread("bin1.jpg", cv.IMREAD_GRAYSCALE)

except Exception as e:
    print("Error loading images:", e)
    # Stop program if images can't be loaded
    exit()

# Check if images were loaded correctly
if im1 is None or im2 is None:
    raise ValueError("Could not load one or both images.")

# Get image sizes
r1, c1 = im1.shape
r2, c2 = im2.shape

# Binary image size constraint (show error )
if r2 > r1 or c2 > c1:
    raise ValueError("Binary image must be smaller than or equal to the grayscale image.")

# Create result image
result = np.zeros((r1, c1), dtype=np.uint8)

# Loop through all pixels of the grayscale image
for y in range(r1):
    for x in range(c1):

        pixvalue = im1[y, x]

        if x < c2 and y < r2:
            # Convert binary pixel to boolean (0 -> False, >0 -> True)
            boolvalue = (im2[y, x] > 0)

            # Hide binary pixel in grayscale pixel
            result[y, x] = steganography(pixvalue, boolvalue)
        else:
            # Outside binary image: keep original pixel
            result[y, x] = pixvalue

# Save the stego image
try:
  cv.imwrite("stego_result.png", result)
  print("Stego image saved")

except Exception as e:
    print("Error saving images:", e)

# -------------------------------
# Retrieve hidden binary image

stego_img = cv.imread("stego_result.png", cv.IMREAD_GRAYSCALE)


r, c = stego_img.shape

# Create an empty binary image
binary = np.zeros((r, c), dtype=np.uint8)

for y in range(r):
    for x in range(c):
          # Use your LSB function to get the hidden bit
          bit = ret_LSB(stego_img[y, x])

          # Convert 0/1 to 0/255 for display
          binary[y, x] = bit * 255


# Save the stego image
try:
  cv.imwrite("retrieveBimg.png", binary)
  print("Stego image saved")

except Exception as e:
    print("Error saving images:", e)





Stego image saved
Stego image saved


**Question 4:**Hide and retrieve three different binary images in an RGB image

In [ ]:
# Paths to images
RGB_path = "rgb.jpg"
bin1_path = "bin1.jpg"
bin2_path = "bin2.bmp"
bin3_path = "bin3.jpg"

try:
  # Load images
  rgbimg = cv.imread(RGB_path)  # RGB image
  b1 = cv.imread(bin1_path, cv.IMREAD_GRAYSCALE)
  b2 = cv.imread(bin2_path, cv.IMREAD_GRAYSCALE)
  b3 = cv.imread(bin3_path, cv.IMREAD_GRAYSCALE)
except Exception as e:
    print("Error loading images:", e)
    # Stop program if images can't be loaded
    exit()



## Copy rgbimg image to preserve original colors
stego = rgbimg.copy()

# Store binary images and their sizes
bins = [b1, b2, b3]
sizes = [b1.shape, b2.shape, b3.shape]

r1, c1, _ = rgbimg.shape

# -------------------------------
# Hide each binary image in R, G, B channels
for y in range(r1):
    for x in range(c1):
        for i in range(3):  # i=0 R, i=1 G, i=2 B
            b = bins[i]
            rows, cols = sizes[i]
            if y < rows and x < cols:
                boolvalue = b[y, x] > 0
                # Use your steganography function to hide the bit
                stego[y, x, i] = steganography(stego[y, x, i], boolvalue)

# Save stego image
try:
  cv.imwrite("stego_rgb.png", stego)
  print("Stego image saved as stego_rgb.png")
except Exception as e:
    print("Error saving images:", e)

# -------------------------------
# Retrieve hidden binary images
retrieved = []

for i in range(3):
    rows, cols = sizes[i]
    bin_img = np.zeros((rows, cols), dtype=np.uint8)
    for y in range(rows):
        for x in range(cols):
            bit = ret_LSB(stego[y, x, i])
            bin_img[y, x] = bit * 255  # 0/1 → 0/255 for visibility
    retrieved.append(bin_img)

# Save retrieved images
try:
  for idx, img in enumerate(retrieved):
      cv.imwrite(f"retrieved_{idx+1}.png", img)
      print(f"Retrieved image {idx+1} saved")
except Exception as e:
    print("Error saving images:", e)


Stego image saved as stego_rgb.png
Retrieved image 1 saved
Retrieved image 2 saved
Retrieved image 3 saved


**Question 5**: Generalize your code to hide multiple bits of data per pixel

In [ ]:
try:
  cover = cv.imread("rgb.jpg", cv.IMREAD_GRAYSCALE)
  secret = cv.imread("pic1.jpg", cv.IMREAD_GRAYSCALE)
except Exception as e:
    print("Error loading images:", e)
    # Stop program if images can't be loaded
    exit()

r1, c1 = cover.shape
r2, c2 = secret.shape

stego = cover.copy()

n_bits = 2  # number of bits to hide per pixel


# Hide n_bits per pixel
for y in range(r1):
    for x in range(c1):
        if y < r2 and x < c2:
            secret_pixel = secret[y, x]
            # Extract n most significant bits from secret_pixel
            for bit_idx in range(n_bits):
                # Shift to get current bit
                bit_to_hide = (secret_pixel >> (7 - bit_idx)) & 1
                # Hide bit using your steganography function
                stego[y, x] = steganography(stego[y, x], bool(bit_to_hide))

try:

  cv.imwrite("stego_multi_bits.png", stego)
  print("Stego image saved")

except Exception as e:
    print("Error saving images:", e)

# -------------------------------
# Retrieve n_bits per pixel
retrieved = np.zeros_like(secret)

for y in range(r2):
    for x in range(c2):
        pixel_value = stego[y, x]
        val = 0
        for bit_idx in range(n_bits):
            # Extract LSB using your ret_LSB()
            bit = ret_LSB(pixel_value)
            val |= bit << (7 - bit_idx)  # reconstruct the hidden pixel
            pixel_value >>= 1  # shift to get next bit
        retrieved[y, x] = val

try:
  cv.imwrite("retrieved_multi_bits.png", retrieved)
  print("Retrieved image saved")

except Exception as e:
  print("Error saving images:", e)


Stego image saved
Retrieved image saved


**Question 6:** Hide a grayscale image in a RGB source image by storing two bits of data in the red channel and
three in the green and blue ones.

In [ ]:
try:
  cover = cv.imread("rgb.jpg")  # RGB image
  secret = cv.imread("pic1.jpg", cv.IMREAD_GRAYSCALE)  # Grayscale secret
except Exception as e:
    print("Error loading images:", e)
    # Stop program if images can't be loaded
    exit()

r1, c1, _ = cover.shape
r2, c2 = secret.shape

stego = cover.copy()

# -------------------------------
# Hide grayscale in RGB using 2-3-3 scheme
for y in range(r2):
    for x in range(c2):
        gray_pixel = secret[y, x]

        # Extract bits for each channel
        r_bits = (gray_pixel >> 6) & 0b11     # 2 bits
        g_bits = (gray_pixel >> 3) & 0b111    # 3 bits
        b_bits = gray_pixel & 0b111           # 3 bits

        # Hide in Red channel
        for i in range(2):
            bit = (r_bits >> (1 - i)) & 1
            stego[y, x, 2] = steganography(stego[y, x, 2], bool(bit))

        # Hide in Green channel
        for i in range(3):
            bit = (g_bits >> (2 - i)) & 1
            stego[y, x, 1] = steganography(stego[y, x, 1], bool(bit))

        # Hide in Blue channel
        for i in range(3):
            bit = (b_bits >> (2 - i)) & 1
            stego[y, x, 0] = steganography(stego[y, x, 0], bool(bit))

try:
  cv.imwrite("stego2.png", stego)
  print("Stego RGB image saved")

except Exception as e:
    print("Error saving images:", e)

Stego RGB image saved


In [ ]:
p=bin(20)
print(p)

0b10100


In [ ]:
bin(0b10100 >> 1)

'0b1010'

In [ ]:
ls

bin1.jpg  retrieveBimg.png  retrieved_multi_bits.png  stego_multi_bits.png
bin2.bmp  retrieved_1.png   rgb.jpg                   stego_result.png
bin3.jpg  retrieved_2.png   sample_data/              stego_rgb.png
pic1.jpg  retrieved_3.png   stego2.png


**Question 7: **Extract and reconstruct the hidden grayscale image.

In [ ]:
# Initialize retrieved grayscale image
retrieved = np.zeros((r2, c2), dtype=np.uint8)

for y in range(r2):
    for x in range(c2):
        # Extract bits from each channel using ret_LSB()
        r_bits = 0
        for i in range(2):
            r_bits |= ret_LSB(stego[y, x, 2]) << (1 - i)
            stego[y, x, 2] >>= 1

        g_bits = 0
        for i in range(3):
            g_bits |= ret_LSB(stego[y, x, 1]) << (2 - i)
            stego[y, x, 1] >>= 1

        b_bits = 0
        for i in range(3):
            b_bits |= ret_LSB(stego[y, x, 0]) << (2 - i)
            stego[y, x, 0] >>= 1

        # Reconstruct grayscale pixel
        retrieved[y, x] = (r_bits << 6) | (g_bits << 3) | b_bits

try:
  cv.imwrite("retrieved_gray.png", retrieved)
  print("Retrieved grayscale image saved")

except Exception as e:
    print("Error saving images:", e)


Retrieved grayscale image saved
